### chapter 35

# 실전 프로젝트 3 : 영어 리뷰 텍스트를 활용한 감성 분석 

## 학습 목표
----
- NLP에서 주로 활용되는 Task에 대해 안다. 
- 허깅페이스에서 데이터셋을 다운로드하여 가져오는 방법에 대해 알아본다.
- 허깅페이스에서 가져온 데이터셋을 전처리하여 커스텀 데이터셋으로 만드는 방법에 대해 알아본다.

### 35-1. NLP의 주요 Task

- 기계 번역(Machine Translation): 한 언어의 텍스트를 다른 언어로 번역합니다.
- 텍스트 요약(Text Summarization): 긴 텍스트를 요약하여 핵심 정보를 제공합니다.
- 질문 응답(Question Answering): 주어진 질문에 대한 답을 텍스트로부터 찾아냅니다.
- 감정 분석(Sentiment Analysis): 텍스트에서 감정이 긍정적인지 부정적인지 판단합니다.

### 35-2. NLP의 데이터셋 만들기

In [ ]:
#PaperswithCode에서 오픈소스 데이터셋 확인 : https://paperswithcode.com/datasets?q=Amazon%20Review%20Full
#Huggingface에서 데이터셋 확인 : https://huggingface.co/datasets/fancyzhx/yelp_polarity

In [13]:
#허깅페이스 라이브러리에서 데이터셋 가져오기
!pip install datasets
!pip install transformers

   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.1 MB 5.6 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.1 MB 5.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.1 MB 4.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/10.1 MB 4.4 MB/s eta 0:00:02
   ------------------- -------------------- 5.0/10.1 MB 4.7 MB/s eta 0:00:02
   ------------------------ --------------- 6.3/10.1 MB 5.0 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.1 MB 5.2 MB/s eta 0:00:01
   ----------------------------------- ---- 8.9/10.1 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.4 MB 6.3 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.4 MB 5.9 MB/s eta 0:00:01
   ------------

In [8]:
from datasets import load_dataset

dataset = load_dataset("fancyzhx/yelp_polarity")

In [9]:
#데이터셋의 구조 확인을 위해 출력하
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})

In [10]:
print(f"Text : {dataset['train'][0]['text']} \n Label : {dataset['train'][0]['label']}")

Text : Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars. 
 Label : 0


### 35-3. 커스텀 데이터셋 만들기

In [11]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import AutoTokenizer

In [12]:
sentence = "Learning NLP in my free time is so useful and funny."

In [13]:
#허깅페이스의 토크나이저를 활용 AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [14]:
#허깅페이스의 토크나이저를 활용 BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [15]:
#"Learning NLP in my free time is so useful and funny."
tokens = tokenizer.tokenize(sentence)

print(tokens)

['learning', 'nl', '##p', 'in', 'my', 'free', 'time', 'is', 'so', 'useful', 'and', 'funny', '.']


In [16]:
tokenizer(sentence)

{'input_ids': [101, 4083, 17953, 2361, 1999, 2026, 2489, 2051, 2003, 2061, 6179, 1998, 6057, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
# 파일 경로와 필요한 파라미터를 지정하여 데이터셋 객체를 생성합니다.
dataset = load_dataset("fancyzhx/yelp_polarity")
train_df = dataset['train'].to_pandas()

In [18]:
train_df

,text,label
0,"Unfortunately, the frustration of being Dr. Go...",0
1,Been going to Dr. Goldberg for over 10 years. ...,1
2,I don't know what Dr. Goldberg was like before...,0
3,I'm writing this review to give you a heads up...,0
4,All the food is great here. But the best thing...,1
...,...,...
559995,Ryan was as good as everyone on yelp has claim...,1
559996,Professional \nFriendly\nOn time AND affordabl...,1
559997,Phone calls always go to voicemail and message...,0
559998,Looks like all of the good reviews have gone t...,0


In [20]:
class TextDataset(Dataset):
    def __init__(self, dataframe, max_len=512):
        self.df = dataframe
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['text']
        label = self.df.iloc[idx]['label']

        # 텍스트를 토크나이징하고 토큰의 ID로 변환합니다. 추가적인 옵션을 사용하여 토큰화를 수행:
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,  # 특수 토큰([CLS], [SEP])을 추가
            max_length=self.max_len,  # 토큰의 최대 길이를 설정
            padding='max_length',     # 최대 길이보다 짧은 시퀀스는 패딩
            return_token_type_ids=True,  # 토큰 타입 ID를 반환 (BERT의 경우 segment IDs)
            truncation=True            # 최대 길이를 초과하는 토큰은 잘라냄
        )
        
        ids = inputs['input_ids']       # 토큰화된 텍스트의 ID
        mask = inputs['attention_mask'] # 어텐션 마스크 (실제 토큰은 1, 패딩은 0)
        
        # 토큰 ID, 어텐션 마스크, 타겟 레이블을 텐서로 변환하여 반환
        return {
            'ids': torch.tensor(ids, dtype=torch.long),         # 입력 ID
            'mask': torch.tensor(mask, dtype=torch.long),       # 어텐션 마스크
            'targets': torch.tensor(label, dtype=torch.long)    # 레이블
        }

In [21]:
dataset = TextDataset(train_df)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
# DataLoader에서 첫 번째 배치 가져오기
data_iter = iter(dataloader)
first_batch = next(data_iter)

# 데이터와 레이블 출력
ids, mask, targets = first_batch['ids'], first_batch['mask'], first_batch['targets']
print(f"ids: {ids} \n mask : {mask} \n targets : {targets}")

In [ ]:
# 데이터 로더를 사용하여 데이터를 반복 처리합니다.
for data in dataloader:
    print(data['ids'].shape, data['mask'].shape, data['targets'].shape)

torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4, 512]) torch.Size([4, 512]) torch.Size([4])
torch.Size([4,